### Gloden cross strategy

In [13]:
import time
from datetime import datetime
import vectorbt as vbt

import pandas as pd
import pandas_ta as ta
import numpy as np
import mplfinance as mpf

### Import csv

In [32]:
data = pd.read_csv('EURUSD_15m.csv')
data.tail()

,timestamp,open,high,low,close,adj close,volumn
11288,2022-04-29 16:45:00+01:00,1.054185,1.054964,1.053963,1.054519,1.054519,0
11289,2022-04-29 17:00:00+01:00,1.053852,1.054185,1.053741,1.053852,1.053852,0
11290,2022-04-29 17:15:00+01:00,1.053852,1.055409,1.053852,1.054519,1.054519,0
11291,2022-04-29 17:30:00+01:00,1.054630,1.054964,1.054296,1.054964,1.054964,0
11292,2022-04-29 17:45:00+01:00,1.054964,1.055409,1.054630,1.055409,1.055409,0


In [33]:
df = data.copy()

### Create strategy

In [6]:
percent_r = ta.Strategy(
    name = 'Willams % R',
    description = 'Willams % R Strategy',
    ta = [
        {'kind': 'ema', 'length': 100},
        {'kind': 'willr', 'length': 20},
        {'kind': 'atr', 'length': 14}
    ]
)
df.ta.strategy(percent_r)

In [7]:
df

,timestamp,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore,EMA_100,WILLR_10,ATRr_14,WILLR_20
0,1514764800000,733.01,737.99,716.80,734.50,8739.23361,1514779199999,6.350664e+06,12593,4586.02898,3.332069e+06,0,NaN,NaN,NaN,NaN
1,1514779200000,734.99,763.55,730.01,751.99,9492.34734,1514793599999,7.108607e+06,13909,5288.53326,3.961098e+06,0,NaN,NaN,NaN,NaN
2,1514793600000,751.77,759.00,730.58,741.01,8939.36851,1514807999999,6.670900e+06,12399,4331.81960,3.233838e+06,0,NaN,NaN,NaN,NaN
3,1514808000000,741.01,752.27,724.15,748.80,11284.08664,1514822399999,8.340628e+06,13681,5950.39223,4.399937e+06,0,NaN,NaN,NaN,NaN
4,1514822400000,748.27,749.98,733.00,746.23,7757.00362,1514836799999,5.760057e+06,11347,3775.43956,2.804330e+06,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9232,1647936000000,2993.00,3027.27,2985.00,3020.74,78938.67390,1647950399999,2.377243e+08,112140,40768.59680,1.227845e+08,0,2771.410375,-14.396521,61.445511,-13.626719
9233,1647950400000,3020.74,3030.15,2988.28,3002.76,83678.34360,1647964799999,2.518801e+08,120628,40285.50780,1.212862e+08,0,2775.991555,-22.457855,60.047260,-21.257002
9234,1647964800000,3002.75,3013.87,2981.00,2984.52,65150.37740,1647979199999,1.956302e+08,89833,32013.61260,9.613750e+07,0,2780.120832,-31.273743,58.106027,-28.997623
9235,1647979200000,2984.52,3016.39,2960.00,2969.64,53494.89220,1647993599999,1.601021e+08,86238,26883.92920,8.048253e+07,0,2783.873686,-46.353964,57.983454,-35.312341


### Create signal¶

In [34]:
class BacktestWilliamsR:
    def __init__(self, params):
        self.orders = pd.DataFrame(columns = ['action', 'open time', 'open', 'close time', 'close', 'T/P', 'S/L', 'result'])
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
        self.unknow_result = []
        self.willr_status = 'idle' # idle, overbought, oversold
        
        self.current_action = 'close'
        self.atr = params['atr']
        self.ema = params['ema']
        self.willr = params['willr']
        
        self.overbought = params['overbought']
        self.oversold = params['oversold']
        
        self.pip_value = params['pip_value']
        self.rr = params['rr']
        self.df = params['df']
        self.plot = params['plot']
        
    def reset_order(self):
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
        
    def get_df(self):
        return self.df
        
    def plot_order(self, order):
        pass
        
    def backtest(self):
        n_bars = 3
        percent_r = ta.Strategy(
            name = 'Willams % R',
            description = 'Willams % R Strategy',
            ta = [
                {'kind': 'ema', 'length': self.ema},
                {'kind': 'willr', 'length': self.willr},
                {'kind': 'atr', 'length': self.atr}
            ]
        )
        self.df.ta.strategy(percent_r)
        self.df = self.df.copy().dropna()
        self.df['timestamp'] = pd.to_datetime(self.df['timestamp'])
        self.df['action'] = None
        
        self.df['entries'] = False
        self.df['exits'] = False
        self.df['short_entries'] = False
        self.df['short_exits'] = False
        self.df['price_action'] = self.df['close']
        
        for index, row in self.df.iterrows():
            entry = 0
            
            if self.current_action == 'close':
                
                if self.willr_status == 'idle':
                    if (row['EMA_'+str(self.ema)] < row['close']) and (row['WILLR_'+str(self.willr)] < self.oversold):
                        self.willr_status = 'oversold'
                        
                    elif (row['EMA_'+str(self.ema)] > row['close']) and (row['WILLR_'+str(self.willr)] > self.overbought):
                        self.willr_status = 'overbought'
                        
                elif self.willr_status == 'oversold':
                    if (row['EMA_'+str(self.ema)] < row['close']) and (row['WILLR_'+str(self.willr)] > -50):
                        self.df.at[index, 'action'] = 'buy'
                        self.df.at[index, 'entries'] = True
                        self.df.at[index, 'price_action'] = row['open']
                        self.order['action'] = 'buy'
                                
                        self.order['open time'] = row['timestamp']
                        self.order['open'] = row['close']
                        self.order['S/L'] = row['open'] - row['ATRr_'+str(self.atr)]
                        self.order['T/P'] = row['open'] + (self.rr * abs(row['open'] - self.order['S/L']))
                        self.current_action = 'buy'
                        
                        self.willr_status = 'idle'
                        
                    elif (row['EMA_'+str(self.ema)] > row['close']):
                        self.willr_status = 'idle'
                        
                elif self.willr_status == 'overbought':
                    if (row['EMA_'+str(self.ema)] > row['close']) and (row['WILLR_'+str(self.willr)] < -50):
                        self.df.at[index, 'action'] = 'sell'
                        self.df.at[index, 'short_entries'] = True
                        self.df.at[index, 'price_action'] = row['open']
                        self.order['action'] = 'sell'
                                
                        self.order['open time'] = row['timestamp']
                        self.order['open'] = row['close']
                        self.order['S/L'] = row['open'] + row['ATRr_'+str(self.atr)]
                        self.order['T/P'] = row['open'] - (self.rr * abs(row['open'] - self.order['S/L']))
                        self.current_action = 'sell'
                        
                        self.willr_status = 'idle'
                        
                    elif (row['EMA_'+str(self.ema)] < row['close']):
                        self.willr_status = 'idle'
        
            if self.current_action == 'buy':
                if row['low'] <= self.order['S/L'] and row['high'] >= self.order['T/P']:
                    self.unknow_result.append(row)
                    
                if row['low'] <= self.order['S/L']:
                    self.df.at[index, 'exits'] = True
                    self.df.at[index, 'price_action'] = row['low']
                    
                    self.order['result'] = 'S/L'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['S/L']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    if self.plot == True: self.plot_order(self.order)
                    self.reset_order()
                    
                elif row['high'] >= self.order['T/P']:
                    self.df.at[index, 'exits'] = True
                    self.df.at[index, 'price_action'] = row['high']
                    
                    self.order['result'] = 'T/P'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['T/P']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    if self.plot == True: self.plot_order(self.order)
                    self.reset_order()

            elif self.current_action == 'sell':
                if row['high'] >= self.order['S/L'] and row['low'] <= self.order['T/P']:
                    self.unknow_result.append(row)
                    
                if row['high'] >= self.order['S/L']:
                    self.df.at[index, 'short_exits'] = True
                    self.df.at[index, 'price_action'] = row['high']
                    
                    self.order['result'] = 'S/L'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['S/L']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    if self.plot == True: self.plot_order(self.order)
                    self.reset_order()
                    
                elif row['low'] <= self.order['T/P']:
                    self.df.at[index, 'short_exits'] = True
                    self.df.at[index, 'price_action'] = row['low']
                    
                    self.order['result'] = 'T/P'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['T/P']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    if self.plot == True: self.plot_order(self.order)
                    self.reset_order()

        try:
            win_rate = round(len(self.orders[self.orders['result'] == 'T/P']) * 100 / len(self.orders), 2)
            loss_rate = round(len(self.orders[self.orders['result'] == 'S/L']) * 100 / len(self.orders), 2)

            win_orders = int((win_rate * len(self.orders)) / 100 )
            loss_orders = len(self.orders) - win_orders
            gain = (win_orders * self.rr) - loss_orders

            result_info = f"total orders: {len(self.orders)}\nema: {self.ema}, willr: {self.willr}, atr: {self.atr}, overbought: {self.overbought}, oversold: {self.oversold}, gain: {gain}\n"
            return {'win_rate': win_rate, 'info': result_info, 'total_order': len(self.orders), 'rr': self.rr, 'gain': gain, 'unknow': self.unknow_result}
        except:
    #         print("No orders to action")
            return {'win_rate': 0, 'info': 'No orders to action'}

In [35]:
pair_test = BacktestWilliamsR({
    'atr': 14,
    'ema': 100,
    'willr': 20,
    'overbought': -20,
    'oversold': -80,
    'pip_value': 0.001,
    'rr': 1.5,
    'plot': False,
    'df': df.copy()
})
pair_test.backtest()

{'win_rate': 60.8,
 'info': 'total orders: 125\nema: 100, willr: 20, atr: 14, overbought: -20, oversold: -80, gain: 65.0\n',
 'total_order': 125,
 'rr': 1.5,
 'gain': 65.0,
 'unknow': []}

In [36]:
"""
Grid parameter example
willr: 10 -> 40
overbought oversold: 70->90, 20->30
"""
def create_grid_params(line1_start, line1_end, line2_start, line2_end):
    params = []
    for i in range(line1_start, line1_end+1):
        for j in range(line2_start, line2_end+1):
            params.append([i, j])
    return params
params_list = create_grid_params(10, 40, 10, 30)
len(params_list)

651

In [37]:
good_params = {'win': 0, 'info': ''}
start = 0
end = 651
win_start = 0

loop_num = start
for param in params_list[start:end]:
    obj = BacktestWilliamsR({'atr': 14, 'ema': 100,  'willr': param[0],'overbought': -1*param[1], 'oversold': -1*(100-param[1]), 'pip_value': 0.001, 'rr': 1.5, 'df': df.copy(), 'plot': False})
    result = obj.backtest()
    if result['win_rate'] > win_start and result['total_order'] > 120:
        win_start = result['win_rate']
        good_params['win'] = result['win_rate']
        good_params['info'] = result['info']
    loop_num = loop_num + 1
    print(f"run: {loop_num} to {end}", end = "\r")
    
print(f"\nwin rate: {good_params['win']} %\n{good_params['info']}")

run: 651 to 651
win rate: 69.29 %
total orders: 127
ema: 100, willr: 24, atr: 14, overbought: -25, oversold: -75, gain: 90.5



### ETHUSDT 15m
* win rate: 79.69 %
* total orders: 256
* ema: 100, willr: 37, atr: 14, overbought: -14, oversold: -86, gain: 254.0

### GBPUSD 15m
* win rate: 73.28 %
* total orders: 131
* ema: 100, willr: 17, atr: 14, overbought: -15, oversold: -85, gain: 106.5

### USDJPY 15m

* win rate: 65.38 %
* total orders: 130
* ema: 100, willr: 12, atr: 14, overbought: -17, oversold: -83, gain: 80.0

### AUDUSD 15m
* win rate: 76.92 %
* total orders: 130
* ema: 100, willr: 29, atr: 14, overbought: -30, oversold: -70, gain: 117.5

### EURUSD 15m
* win rate: 69.29 %
* total orders: 127
* ema: 100, willr: 24, atr: 14, overbought: -25, oversold: -75, gain: 90.5